
# Buy Now, Pay Later (BNPL)

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/fsi/credit_decisioning/fsi-credit-decisioning-ml-3.png" style="float: right" width="800px">


### *"Can we allow a user without sufficient balance in their account to complete the current (debit or credit) transaction?"*


We will utilize our credit risk model (built in the previous steps) in real-time to answer this question.

The payment system will be able to call our API in realtime and get a score within a few ms. 

With this information, we'll be able to offer our customer the choice to pay with a credit automatically, or refuse if the model believes the risk is too high and  will likely result in a payment default.


These types of decisions are typically embedded in live Point Of Sales (stores, online shop). That is why we need real-time serving capabilities.


<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=1444828305810485&notebook=%2F03-Data-Science-ML%2F03.4-model-serving-BNPL-credit-decisioning&demo_name=lakehouse-fsi-credit&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-fsi-credit%2F03-Data-Science-ML%2F03.4-model-serving-BNPL-credit-decisioning&version=1&user_hash=7804490f0d3be4559d29a7b52959f461489c4ee5e35d4afc7b55f311360ac589">

### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-lakehouse-fsi-credit-junyi_tiong` from the dropdown menu ([open cluster configuration](https://e2-demo-field-eng.cloud.databricks.com/#setting/clusters/0922-083237-e7fg83pu/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('lakehouse-fsi-credit')` or re-install the demo: `dbdemos.install('lakehouse-fsi-credit')`*

# Deploying the Credit Scoring model for real-time serving


Let's deploy our model behind a scalable API to evaluate credit-worthiness in real-time.

## Databricks Model Serving

Now that our model has been created with Databricks AutoML, we can easily flag it as Production Ready and turn on Databricks Model Serving.

We'll be able to send HTTP Requests and get inference in real-time.

Databricks Model Serving is fully serverless:

* One-click deployment. Databricks will handle scalability, providing blazing fast inferences and startup time.
* Scale down to zero as an option for best TCO (will shut down if the endpoint isn't used).
* Built-in support for multiple models & version deployed.
* A/B Testing and easy upgrade, routing traffic between each versions while measuring impact.
* Built-in metrics & monitoring.

In [0]:
%pip install mlflow==2.19.0
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

In [0]:
import mlflow
model_name = "dbdemos_fsi_credit_decisioning"
mlflow.set_registry_uri('databricks-uc')

In [0]:

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ServedEntityInput, EndpointCoreConfigInput, AutoCaptureConfigInput

model_name = f"{catalog}.{db}.dbdemos_fsi_credit_decisioning"
serving_endpoint_name = "dbdemos_fsi_credit_decisioning_endpoint"
w = WorkspaceClient()
endpoint_config = EndpointCoreConfigInput(
    name=serving_endpoint_name,
    served_entities=[
        ServedEntityInput(
            entity_name=model_name,
            entity_version=get_latest_model_version(model_name),
            scale_to_zero_enabled=True,
            workload_size="Small"
        )
    ],
    auto_capture_config = AutoCaptureConfigInput(catalog_name=catalog, schema_name=db, enabled=True, table_name_prefix="inference_table" )
)

force_update = False #Set this to True to release a newer version (the demo won't update the endpoint to a newer model version by default)
existing_endpoint = next((e for e in w.serving_endpoints.list() if e.name == serving_endpoint_name), None)
if existing_endpoint == None:
    print(f"Creating the endpoint {serving_endpoint_name}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.create_and_wait(name=serving_endpoint_name, config=endpoint_config)
else:
  print(f"endpoint {serving_endpoint_name} already exist...")
  if force_update:
    w.serving_endpoints.update_config_and_wait(served_entities=endpoint_config.served_entities, name=serving_endpoint_name)
    

Our model endpoint was automatically created. 

Open the [endpoint UI](#mlflow/endpoints/dbdemos_fsi_credit_decisioning_endpoint) to explore your endpoint and use the UI to send queries.

*Note that the first deployment will build your model image and take a few minutes. It'll then stop & start instantly.*


### Testing the model

Now that the model is deployed, let's test it with information for a customer trying to temporarily increase their credit limits or obtain a micro-loan at the point-of-sale.

In [0]:
from mlflow.store.artifact.models_artifact_repo import ModelsArtifactRepository
from mlflow.models.model import Model

p = ModelsArtifactRepository(f"models:/{model_name}@prod").download_artifacts("") 
dataset =  {"dataframe_split": Model.load(p).load_input_example(p).to_dict(orient='split')}

In [0]:
import mlflow
from mlflow import deployments
client = mlflow.deployments.get_deploy_client("databricks")
predictions = client.predict(endpoint=serving_endpoint_name, inputs=dataset)

prediction_score = list(predictions["predictions"])[0]
print(
    f"The transaction will be approved. Score: {prediction_score}."
    if prediction_score == 0
    else f"The transaction will not be approved. Score: {prediction_score}."
)

# Updating your model and monitoring its performance with A/B testing 


<img style="float: right;" src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/fsi/fraud-detection/model-serving-ab-testing.png" width="850px" />

Databricks Model Serving let you easily deploy & test new versions of your model.

You can dynamically reconfigure your endpoint to route a subset of your traffic to a newer version. In addition, you can leverage endpoint monitoring to understand your model behavior and track your A/B deployment.

* Without making any production outage
* Slowly routing requests to the new model
* Supporting auto-scaling & potential bursts
* Performing some A/B testing ensuring the new model is providing better outcomes
* Monitorig our model outcome and technical metrics (CPU/load etc)

Databricks makes this process super simple with Serverless Model Serving endpoint.


<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=lakehouse&org_id=1444828305810485&notebook=%2F03-Data-Science-ML%2F03.4-model-serving-BNPL-credit-decisioning&demo_name=lakehouse-fsi-credit&event=VIEW&path=%2F_dbdemos%2Flakehouse%2Flakehouse-fsi-credit%2F03-Data-Science-ML%2F03.4-model-serving-BNPL-credit-decisioning&version=1&user_hash=7804490f0d3be4559d29a7b52959f461489c4ee5e35d4afc7b55f311360ac589">


## Model monitoring and A/B testing analysis

Because the Model Serving runs within our Lakehouse, Databricks will automatically save and track all our Model Endpoint results as a Delta Table.

We can then easily plug a feedback loop to start analysing the revenue in $ each model is offering. 

All these metrics, including A/B testing validation (p-values etc) can then be pluged into a Model Monitoring Dashboard and alerts can be sent for errors, potentially triggering new model retraining or programatically updating the Endpoint routes to fallback to another model.


<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/fsi/fraud-detection/model-serving-monitoring.png" width="1200px" />


## Next

Making sure your model is fair towards customers of any demographics are extremely important parts of building production-ready ML models for FSI use cases. <br/>
Explore your model with [03.5-Explainability-and-Fairness-credit-decisioning]($./03.5-Explainability-and-Fairness-credit-decisioning) on the Lakehouse.

## Conclusion: the power of the Lakehouse

In this demo, we've seen an end 2 end flow with the Lakehouse:

- Data ingestion made simple with Delta Live Table
- Leveraging Databricks warehouse to making credit decisions
- Model Training with AutoML for citizen Data Scientist
- Ability to tune our model for better results, improving our revenue
- Ultimately, the ability to deploy and make explainable ML predictions, made possible with the full Lakehouse capabilities.

[Go back to the introduction]($../00-Credit-Decisioning) or discover how to use Databricks Workflow to orchestrate everything together through the [05-Workflow-Orchestration-credit-decisioning]($../05-Workflow-Orchestration/05-Workflow-Orchestration-credit-decisioning).

In [0]:
print("Hello World")